In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook
import seaborn as sns
import gc
import plotly.graph_objects as go

# /kaggle/input/ashrae-energy-prediction/weather_test.csv
# /kaggle/input/ashrae-energy-prediction/building_metadata.csv
# /kaggle/input/ashrae-energy-prediction/train.csv
# /kaggle/input/ashrae-energy-prediction/test.csv
# /kaggle/input/ashrae-energy-prediction/sample_submission.csv
# /kaggle/input/ashrae-energy-prediction/weather_train.csv

In [ ]:
# Import datasets For train

train_df = pd.read_csv('/kaggle/input/ashrae-energy-prediction/train.csv')
weather_train_df = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_train.csv')
building_meta_df = pd.read_csv('/kaggle/input/ashrae-energy-prediction/building_metadata.csv')

# Import datasets For test

# test_df = pd.read_csv('/kaggle/input/ashrae-energy-prediction/test.csv')
# weather_test_df = pd.read_csv('/kaggle/input/ashrae-energy-prediction/weather_test.csv')
# sample_submission = pd.read_csv('/kaggle/input/ashrae-energy-prediction/sample_submission.csv')


In [ ]:
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage_2(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if col == 'timestamp': continue
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df

# What is it all about?

in this kernel for this special competition, We are working on predicting the energy consumption in KwH<br>

For now We just check train data


![shifting trends of global energy usage](https://cdn.borntoengineer.com/2018/07/1-Header.jpg)

[image source ](www.borntoengineer.com)

In [ ]:
train_df = reduce_mem_usage_2(train_df ,use_float16=True)
weather_train_df = reduce_mem_usage_2(weather_train_df ,use_float16=True)
building_meta_df = reduce_mem_usage_2(building_meta_df ,use_float16=True)

In [ ]:
train_df = train_df.merge(building_meta_df, on='building_id', how='left')
train_df = train_df.merge(weather_train_df, on=['site_id', 'timestamp'], how='left')
del weather_train_df, building_meta_df
gc.collect();

In [ ]:
train_df.timestamp = pd.to_datetime(train_df.timestamp)

In [ ]:
train_df['date'] = train_df.timestamp.dt.date
train_df ['hour'] = train_df.timestamp.dt.hour

# What we want to check? (Goals) : 

1- ) **Linear Corellation check** : At least there must be a linear correlation between meter_reading and other features.

2-) **Nan Check** : this is not hypothesis but. There must be no so much NAN in train and test. But may have in metadata and weather.

3-) **Time affect on meter_reading** : is monthes of the year affect the amount of energy they use?

4-) **Different Type of building** :  Let's see different type of buildings. By the type we can use different Feature Engineerings.

5-) **Energy Usage By Building Type** :  I think we must have most energy usage by Education buildings. Let's check what about others.

6-) **Mode in Energy Meter Type**

7-) **Energy_Type Usage Count** :  let's see which energy type is used in most buildings

8-) **Energy_Type Usage Count for each primary_use** :  for each primary use, how many of that primary use buildings are using different energy types.

8.1-) **meter_reading of different energy sources for each primary use** :  KWH energy usage by different energy types in each primary use.
       
       as we can see here steam is eating money and energy

9-) **Mean Of primary_use And year_built and square feet**  :  check some numeric data

10-) **Noise Detection by hand** :  Let's remove noises by hand




# Resuluts :

1-) There was no meaningful relation .

2-) That was right for train. that was more right in test :|. we may have challanges for feature engineering and data filling.

One usefull way is to find mean of each building type ( primary_ use ) and fill with them for Floor_count and others.

3-) It seems time is not main reason for indcreasing and decreasing of energy  

4-) becuase of memory problems I'll find this answer later.

5-) Education has the most building. After education : Public Other Office Lodging residental.

6- Education has most energy usage. after education we have Office Entertainment Service and Lodging residental

7-) Most of the buildings has Electrity. But does electricity use most energy? ( NO!!! we will see later).

8-) As we can see most of the buildings are using electricity in each primary use

8.1-) Steam is dominating here. so We can see one very important feature here. STEAM.

9-) we saw some statistic data.

10-) we found that we have very very much noise in our data. and we can remove them by hand too. not so hard. but need some time.

In [ ]:
# just first Let's see 5 first rows. this is my habit.
print(f'shape is {train_df.shape}')
train_df.head()

## Linear Corellation check

In [ ]:
# check first hypothesis :
corr = train_df.corr()
# Don;t show upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, mask=mask)

weird!!!! No Sensible linear relation between columns.

# Nan check

In [ ]:
train_df.isnull().sum() * 100 / train_df.shape[0]

For **TRAIN** : 
So we see some columns have many Nan Values.<br>
So bad we don't have all floor_counts | year_built . these two variable are super important( At least I think )



# Time affect on meter_reading

In [ ]:
fig, axes = plt.subplots(figsize=(14,6))
train_df[['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, label='By hour', alpha=0.8).set_ylabel('Meter reading', fontsize=14)

We can see 2 peak here

This notebook ( https://www.kaggle.com/nroman/eda-for-ashrae ) shows why we have these 2 peaks.

# **Different Type of building** 

In [ ]:
ax = sns.barplot(train_df.groupby(['primary_use']).size().reset_index(name='counts')['primary_use'], train_df.groupby(['primary_use']).size().reset_index(name='counts')['counts'])
ax.set(xlabel='Primary Usage', ylabel='# of records', title='Primary Usage vs. # of records')
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
plt.show()

#  **Energy Usage By Building Type** 

In [ ]:
ax = sns.barplot(train_df.groupby('primary_use').meter_reading.agg('sum').reset_index()['primary_use'],
                train_df.groupby('primary_use').meter_reading.agg('sum').reset_index()['meter_reading'])
ax.set(xlabel='Primary Usage', ylabel='# of records', title='Primary Usage vs. # of records')
ax.set_xticklabels(ax.get_xticklabels(), rotation=50, ha="right")
plt.show()

# Mode in Energy Meter Type

In [ ]:
train_df.groupby('primary_use').meter.agg(pd.Series.mode).reset_index()

# Energy_Type Usage Count

let's see which energy type is used in most buildings

In [ ]:
ax = sns.barplot(['electricity', 'chiledwater', 'steam', 'hotwater'], train_df.meter.value_counts().to_frame()['meter'])
ax.set(xlabel='meter Types', ylabel='How much is used', title='meter Types vs. # of usage')

# Energy_Type Count for each primary_use

check type of energy each primary_use has for it's buildings.

In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=4)
fig.set_figheight(12)
fig.set_figwidth(15)
# fig.tight_layout()
fig.subplots_adjust(wspace=0.5)
fig.subplots_adjust(hspace=0.5)
primary_uses = train_df.primary_use.unique()

for i, primary_use in enumerate(primary_uses):
    sns.countplot(x="meter", data = train_df[train_df.primary_use  == primary_use],ax=axes[i//4,i%4]).title.set_text(str(primary_use))



# meter_reading of different energy sources for each primary use 

KWH energy usage for each energy type for each primay_use

In [ ]:
meter_types = { 0 : 'electricity', 1 : 'chilledwater', 2 : 'steam', 3 : 'hotwater' }


fig, axes = plt.subplots(nrows=4, ncols=4)
fig.set_figheight(12)
fig.set_figwidth(15)
# fig.tight_layout()
fig.subplots_adjust(wspace=0.5)
fig.subplots_adjust(hspace=0.5)
primary_uses = train_df.primary_use.unique()

for i, primary_use in enumerate(primary_uses):
    sns.barplot(train_df[train_df.primary_use == primary_use].groupby('meter').meter_reading.sum(),
               np.vectorize(meter_types.get)(train_df[train_df.primary_use == primary_use].groupby(['meter'])['meter_reading'].mean().keys()),
                ax=axes[i//4,i%4]).title.set_text(str(primary_use))


STEAM. reasonable but was amazing to know steam is using this much energy.

# Mean Of square_feet

In [ ]:
train_df.groupby('primary_use').square_feet.agg(['mean'])

# Mean Of year_built

In [ ]:
train_df.groupby('primary_use').year_built.agg(['mean'])

Nothing for services. What a huge missing data.

# Mean Of meter_reading by primary_use

In [ ]:
train_df.groupby('primary_use').meter_reading.agg(['mean'])

Until here we have 2 suspect : EDUCATION AND STEAM


# Noise Detection By hand 

We have noises in data. To detect these noises, we need to see buildings and check.

I think when we have sequential zeros for meter_reading ( for example 15 days. ) that is noise.

I'll show just one sample. I'll create another notebook for noise detection using EDA

Show Site ids

In [ ]:
fig, axes = plt.subplots(8,2,figsize=(14, 30), dpi=100)

for i in range(train_df['site_id'].nunique()):
    train_df[train_df.site_id == i][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax = axes[i%8][i//8])
    axes[i%8][i//8].legend();
    axes[i%8][i//8].set_title('site_id {}'.format(i), fontsize=13);
    plt.subplots_adjust(hspace=0.45)

Show primary uses of one site id

In [ ]:
fig, axes = plt.subplots(8,2,figsize=(14, 30), dpi=100)
for i, use in enumerate(train_df['primary_use'].value_counts().index.to_list()):
    try:
        train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == use)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
        train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == use)][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[i%8][i//8], alpha=1, label='By day', color='tab:orange').set_xlabel('');
        axes[i%8][i//8].legend();
    except TypeError:
        pass
    axes[i%8][i//8].set_title(use, fontsize=13);
    plt.subplots_adjust(hspace=0.45)

 Now we just choose, site_id 13.
 let's check 

In [ ]:
train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education')][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot()

check this special site id and primary use energy type usage

In [ ]:
meter_types = { 0 : 'electricity', 1 : 'chilledwater', 2 : 'steam', 3 : 'hotwater' }
country=train_df[(train_df.site_id == 13) & (train_df.primary_use == 'Education') ].groupby('meter')['meter_reading'].sum()

fig = go.Figure(go.Treemap(
    labels=np.vectorize(meter_types.get)(country.keys()),
    parents = ["Total Energy Usage Type "]*len(country),
    values =  country
))

fig.show()

Now let's see buildings in this site id and this special primary use

In [ ]:

fig, axes = plt.subplots(9,2,figsize=(14, 36), dpi=100)
for i, building in enumerate(train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == 2)]['building_id'].value_counts(dropna=False).index.to_list()):
    train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == 2) & (train_df['building_id'] == building)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i%9][i//9], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    train_df[(train_df['site_id'] == 13) & (train_df['primary_use'] == 'Education') & (train_df['meter'] == 2) & (train_df['building_id'] == building)][['timestamp', 'meter_reading']].set_index('timestamp').resample('D').mean()['meter_reading'].plot(ax=axes[i%9][i//9], alpha=1, label='By day', color='tab:orange').set_xlabel('');
    axes[i%9][i//9].legend();
    axes[i%9][i//9].set_title('building_id: ' + str(building), fontsize=13);
    plt.subplots_adjust(hspace=0.45)

So. if you want to delete all the noises you must do these last parts for every site id and pirmary use. then remove those zeros.

That's it.